# ♻️ Garbage Classification Project

This notebook implements a Deep Learning model to classify waste into different categories. We will use Transfer Learning with VGG16.

## 1. Authentication & Setup
We start by configuring the environment and copying the Kaggle API key to download the dataset programmatically.

In [ ]:
cp "/content/drive/MyDrive/Colab Notebooks/Luminar vscode adichpoyii/Deep learning/kaggle.json" /content/

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

## 2. Import Basic Libraries
Importing `numpy` for linear algebra operations and `matplotlib` for plotting images and graphs.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 3. Download Dataset
Downloading the [Garbage Classification dataset](https://www.kaggle.com/datasets/mostafaabla/garbage-classification) from Kaggle.

In [ ]:
!kaggle datasets download -d "mostafaabla/garbage-classification"

## 4. Extract Dataset
Unzipping the downloaded file to access the image data.

In [ ]:
!unzip /content/garbage-classification.zip

## 5. Import Deep Learning Libraries
We use **TensorFlow/Keras** for building the CNN model. We import:
*   `VGG16`: The pre-trained base model.
*   `Sequential`: To stack layers linearly.
*   `ImageDataGenerator`: For data augmentation and preprocessing.

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## 6. Preprocessing & Augmentation
We create an `ImageDataGenerator` to:
1.  **Rescale**: Normalize pixel values from 0-255 to 0-1 (by dividing by 255).
2.  **Split**: Reserve 30% of data for validation.

In [ ]:
gen = ImageDataGenerator(1./255,validation_split=.3)

## 7. Load Image Data
Creating generators for Training and Validation sets:
*   Target size: **224x224** (Required for VGG16)
*   Batch size: **32**
*   Class mode: **Categorical** (since we have multiple waste classes)

In [ ]:
train_data = gen.flow_from_directory(directory="/content/garbage_classification",
                                     class_mode="categorical",
                                     seed = 42,
                                     shuffle =True,
                                     subset = "training",
                                     batch_size = 32,
                                     target_size=(224, 224))
vaild_data = gen.flow_from_directory(directory="/content/garbage_classification",
                                     class_mode="categorical",
                                     seed = 42,
                                     shuffle =True,
                                     subset = "validation",
                                     batch_size = 32,
                                     target_size=(224, 224))

## 8. Initialize Base Model (VGG16)
Downloading VGG16 with `imagenet` weights. We set `include_top=False` to remove the final classification layers, as we will add our own for garbage classes. We calculate `trainable = False` to freeze these layers.

In [ ]:
base_model = VGG16(include_top=False,
                   weights="imagenet",
                   input_shape = (224,224,3))
base_model.trainable = False

## 9. Define Model Architecture
We stack:
1.  **Base Model**: VGG16 feature extractor.
2.  **Flatten**: To convert 2D feature maps to 1D vector.
3.  **Dense (128)**: Hidden layer with ReLU activation.
4.  **Dense (12)**: Output layer with Softmax activation (12 classes).

In [ ]:
from tensorflow.keras.layers import Dense,Flatten,Dropout

In [ ]:
model = Sequential([base_model,
                   Flatten(),
                   Dense(128,activation="relu"),

                   Dense(12,activation="softmax")])
model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

## 10. Train the Model
Training for 10 epochs using the training and validation generators.

In [ ]:
model.fit(train_data,epochs=10,validation_data=vaild_data)

## 11. Class Labels
Indexing the class labels to map predictions back to class names.

In [ ]:
classes =train_data.class_indices

In [ ]:
clas = dict(zip(classes.values(),classes.keys()))

## 12. Prediction Pipeline
Functions to load, preprocess, and classify new images.

In [ ]:
import cv2

In [ ]:
img_arr = cv2.imread("/content/download.jpg")

In [ ]:
img_arr.shape

In [ ]:

img_arr = cv2.cvtColor(img_arr,cv2.COLOR_BGR2RGB)
imge_resize = cv2.resize(img_arr,(224,224))

In [ ]:
imge_resize.shape

In [ ]:
imge_reshaped = imge_resize.reshape(1,224,224,3)/255
y_pred = model.predict(imge_reshaped)
ypred = np.argmax(y_pred)
clas[int(ypred)]

In [ ]:
def find(path):
  img = cv2.imread(path)
  img_arr = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
  plt.imshow(img_arr)
  plt.axis("off")
  plt.show()
  imge_resize = cv2.resize(img_arr,(224,224))
  imge_reshaped = imge_resize.reshape(1,224,224,3)/255
  y_pred = model.predict(imge_reshaped)
  print(y_pred)
  ypred = np.argmax(y_pred)
  print()
  print(ypred)
  return clas[int(ypred)]

In [150]:
find("/content/images.jpg")